In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
# data load

titanic_df = pd.read_csv('titanic_train.csv')


In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [4]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
#전처리
#passengerID, name, 

In [17]:
#결측치 age, cabin, embarked

titanic_df['Age'] = titanic_df['Age'].fillna( titanic_df['Age'].mean())
titanic_df['Cabin'] = titanic_df['Cabin'].fillna( 'N')
titanic_df['Embarked'] = titanic_df['Embarked'].fillna( 'N')

titanic_df.isnull().sum().sum()

np.int64(0)

In [18]:
titanic_df = titanic_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,N,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,N,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,N,S


In [19]:
# 라벨 인코딩 - 범주형
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
titanic_df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,N,S
1,1,1,female,38.0,1,0,71.2833,C,C


In [20]:
col_names = ['Cabin', 'Sex', 'Embarked']
for col in col_names:
    le = LabelEncoder()
    le.fit(titanic_df[col])
    titanic_df[col] = le.transform(titanic_df[col])

titanic_df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,22.0,1,0,7.2500,7,3
1,1,1,0,38.0,1,0,71.2833,2,0


In [21]:

X_titanic_df = titanic_df.drop('Survived', axis=1)
y_titanic_df = titanic_df['Survived']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11)

In [23]:
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

In [24]:
dt_clf.fit(X_train, y_train )
rf_clf.fit(X_train, y_train)
lr_clf.fit(X_train, y_train)

dt_pred = dt_clf.predict(X_test )
rf_pred = rf_clf.predict(X_test)
lr_pred = lr_clf.predict(X_test)

accuracy_score(y_test, dt_pred), accuracy_score(y_test, rf_pred), accuracy_score(y_test, lr_pred)

(0.7877094972067039, 0.8547486033519553, 0.8659217877094972)

In [25]:
#교차 검증
dt_clf_cv = DecisionTreeClassifier(random_state=11)
rf_clf_cv = RandomForestClassifier(random_state=11)
lr_clf_cv = LogisticRegression(solver='liblinear')

In [26]:
kfold = KFold(n_splits=5) #stratified
scores = []

for train_index, test_index in kfold.split(X_titanic_df) :
    X_train, X_test = X_titanic_df.values[train_index], X_titanic_df.values[test_index]
    y_train, y_test = y_titanic_df.values[train_index], y_titanic_df.values[test_index]

    dt_clf_cv.fit(X_train, y_train )
    dt_pred = dt_clf_cv.predict(X_test )
    scores.append(accuracy_score(y_test, dt_pred))
   
print(scores)
print('평균 정확도 : ', np.mean(scores))

[0.7541899441340782, 0.7808988764044944, 0.7865168539325843, 0.7696629213483146, 0.8202247191011236]
평균 정확도 :  0.782298662984119


In [27]:
#함수로 만들기 cross_val_score()

dt_clf_gcv = DecisionTreeClassifier(random_state=11)
rf_clf_gcv = RandomForestClassifier(random_state=11)
lr_clf_gcv = LogisticRegression(solver='liblinear')



In [28]:
#파라미터 튜닝 + 교차검증
#DecisionTree parameter, 
parameters = { 'max_depth':[1,2,3], 'min_samples_split':[2,3] }
grid_dt_clf = GridSearchCV(dt_clf_gcv, param_grid= parameters, scoring='accuracy', cv=5)
grid_dt_clf.fit(X_train, y_train)

score_df = pd.DataFrame(grid_dt_clf.cv_results_)
score_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001298,0.001327,0.001125,0.000695,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.804196,0.762238,0.825175,0.774648,0.746479,0.782547,0.028501,3
1,0.000999,0.000004,0.000418,0.000512,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.804196,0.762238,0.825175,0.774648,0.746479,0.782547,0.028501,3
2,0.000835,0.000798,0.000544,0.000501,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.748252,0.762238,0.825175,0.774648,0.746479,0.771358,0.028786,5
3,0.000840,0.000422,0.000409,0.000501,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.748252,0.762238,0.825175,0.774648,0.746479,0.771358,0.028786,5
4,0.001170,0.000181,0.000448,0.000549,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.797203,0.804196,0.818182,0.795775,0.774648,0.798001,0.014118,1
5,0.001241,0.000206,0.000399,0.000489,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.797203,0.804196,0.818182,0.795775,0.774648,0.798001,0.014118,1


In [29]:
best_dt_clf = grid_dt_clf.best_estimator_
best_dt_pred = best_dt_clf.predict(X_test)
accuracy_score(y_test, best_dt_pred)

0.8426966292134831

In [ ]:
#평가 & 정확도 높이기 - 튜닝
# 스케일링
# 속성선택
# model : RF, LR
# 파라미터터

1. 랜덤 포레스트 파라미터 튜닝
주요 파라미터를 일부 수정해 모델 성능을 높이는 방법
기본값 대비 조금만 조정해보는 접근

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# RandomForest 기본값 수정
rf_clf = RandomForestClassifier(n_estimators=200, max_depth=8, random_state=11)

# 학습 및 예측
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)

# 성능 확인
print("Random Forest Accuracy (튜닝 후):", accuracy_score(y_test, rf_pred))


Random Forest Accuracy (튜닝 후): 0.8595505617977528


2. 로지스틱 회귀 간단히 튜닝
c값을 조금 변경해보는 방식식

In [31]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression 튜닝
lr_clf = LogisticRegression(C=0.5, solver='liblinear', random_state=11)

# 학습 및 예측
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)

# 성능 확인
print("Logistic Regression Accuracy (튜닝 후):", accuracy_score(y_test, lr_pred))


Logistic Regression Accuracy (튜닝 후): 0.8370786516853933


3. 결정트리 간단히 튜닝하기
트리의 깊이를 제한하거나 분할 기준을 조정

In [32]:
from sklearn.tree import DecisionTreeClassifier

# Decision Tree 튜닝
dt_clf = DecisionTreeClassifier(max_depth=5, min_samples_split=4, random_state=11)

# 학습 및 예측
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)

# 성능 확인
print("Decision Tree Accuracy (튜닝 후):", accuracy_score(y_test, dt_pred))


Decision Tree Accuracy (튜닝 후): 0.8595505617977528


In [34]:
print("튜닝 후 결과:")
print(f"Random Forest Accuracy: {accuracy_score(y_test, rf_pred):.4f}")
print(f"Logistic Regression Accuracy: {accuracy_score(y_test, lr_pred):.4f}")
print(f"Decision Tree Accuracy: {accuracy_score(y_test, dt_pred):.4f}")


튜닝 후 결과:
Random Forest Accuracy: 0.8596
Logistic Regression Accuracy: 0.8371
Decision Tree Accuracy: 0.8596
